In [1]:
import os
import scanpy as sc
import pandas as pd
import numpy as np
from tqdm import tqdm
os.chdir('/lustre/scratch117/cellgen/team205/cs42/VDJ_collab_manuscript/')
sc.logging.print_header()

scanpy==1.9.1 anndata==0.8.0 umap==0.4.6 numpy==1.19.4 scipy==1.5.3 pandas==1.4.4 scikit-learn==1.1.2 statsmodels==0.13.2 python-igraph==0.8.3 louvain==0.7.0 leidenalg==0.8.3


In [2]:
import dandelion as ddl
ddl.logging.print_header()

dandelion==0.3.1.dev21 pandas==1.4.4 numpy==1.19.4 matplotlib==3.5.2 networkx==2.8.8 scipy==1.5.3


In [3]:
# downloaded from https://developmental.cellatlas.io/fetal-immune
adata = sc.read_h5ad('/nfs/team205/ed6/data/Fetal_immune/PAN.A01.v01.entire_data_normalised_log.20210429.LYMPHOID.embedding.h5ad')
adata

AnnData object with n_obs × n_vars = 241950 × 33538
    obs: 'n_counts', 'n_genes', 'file', 'mito', 'doublet_scores', 'predicted_doublets', 'name', 'uniform_label', 'uniform_label_expanded_merged', 'uniform_label_lvl0', 'organ', 'Sample.lanes', 'Sort_id', 'age', 'method', 'donor', 'sex', 'Processing_method', 'AnnatomicalPart', 'Sample', 'bbk', 'scvi_clusters', 'anno_lvl_2_LYMPHOID', 'anno_lvl_2_MYELOID', 'anno_lvl_2_MEM_PROGENITORS', 'leiden_150'
    var: 'GeneID', 'GeneName', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'anno_lvl_2_LYMPHOID_colors', 'dendrogram_scvi_clusters', 'donor_colors', 'isin_HSC_IMMUNE_colors', 'isin_LYMPHOID_colors', 'isin_MEM_PROGENITORS_colors', 'isin_MYELOID_LYMPHOID_colors', 'isin_MYELOID_colors', 'isin_STROMA_colors', 'leiden_150_pred_label_expanded_colors', 'organ_colors', 'scvi', 'scvi_clusters_colors', 'uniform_label_expanded_merged_propagated_colors'
    obsm: 'X_scvi', 'X_umap'
    obsp: 'scvi_connectivities', 'scvi_distance

update annotation with anno_lvl_2_final_clean

In [4]:
# downloaded from https://developmental.cellatlas.io/fetal-immune
obs = pd.read_csv('/nfs/team205/ed6/data/Fetal_immune/PAN.A01.v01.entire_data_normalised_log.20210429.full_obs.annotated.clean.csv', index_col = 0) # final annotations and removed maternal contaminant cells

<ipython-input-4-aa4fc4311cad>:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.


In [5]:
adata.obs['anno_lvl_2_final_clean'] = np.nan
adata.obs['anno_lvl_2_final_clean'].update(obs['anno_lvl_2_final_clean'])

In [6]:
#vdj_loc = '/lustre/scratch117/cellgen/team205/sharedData/cs42/all-dandelion-paper-map'
# new address after updating leftmost J gene
vdj_loc = '/lustre/scratch117/cellgen/team205/cs42/VDJ_collab_manuscript/data_remap_11012023/dandelion-remap'

## tcr-ab

In [7]:
# read in metadata
trab_meta = pd.read_csv('metadata/abTCR_metadata_cleaned_24032022.csv')
trab_meta

,lane_id,GEX_id
0,FCAImmP7292944,FCAImmP7292031
1,FCAImmP7292941,FCAImmP7292028
2,FCAImmP7292942,FCAImmP7292029
3,FCAImmP7292943,FCAImmP7292030
4,FCAImmP7292947,FCAImmP7292035
...,...,...
60,Human_colon_16S8157851,Human_colon_16S8159186
61,Human_colon_16S8157852,Human_colon_16S8159187
62,Human_colon_16S8157853,Human_colon_16S8159188
63,Human_colon_16S8157854,Human_colon_16S8159189


In [8]:
tcrab = {}
for x, y in tqdm(zip(trab_meta['lane_id'], trab_meta['GEX_id'])):
    file = vdj_loc + '/'+x+'/dandelion/tmp/all_contig_igblast_db-all.tsv'
    try:
        tmp = pd.read_csv(file, sep ='\t')
        tmp['cell_id'] = [y + '-' + z.split('-1')[0] for z in tmp['cell_id']]
        tmp = tmp[tmp['cell_id'].isin(adata.obs_names)].copy()
        if 'duplicate_count' not in tmp:
            tmp['duplicate_count'] = tmp['umi_count']
            tmp.drop('umi_count', axis = 1, inplace = True)
        tcrab[x] = ddl.Dandelion(tmp)
    except:
        # print(x)
        pass
len(tcrab)

65it [01:50,  1.70s/it]


61

In [9]:
# concatenate
tcrab = ddl.concat([tcrab[x] for x in tcrab], prefixes = list(tcrab.keys()))
tcrab

Dandelion class object with n_obs = 65194 and n_contigs = 169396
    data: 'sequence_id', 'sequence', 'rev_comp', 'productive', 'v_call', 'd_call', 'j_call', 'sequence_alignment', 'germline_alignment', 'junction', 'junction_aa', 'v_cigar', 'd_cigar', 'j_cigar', 'stop_codon', 'vj_in_frame', 'locus', 'c_call', 'junction_length', 'np1_length', 'np2_length', 'v_sequence_start', 'v_sequence_end', 'v_germline_start', 'v_germline_end', 'd_sequence_start', 'd_sequence_end', 'd_germline_start', 'd_germline_end', 'j_sequence_start', 'j_sequence_end', 'j_germline_start', 'j_germline_end', 'v_score', 'v_identity', 'v_support', 'd_score', 'd_identity', 'd_support', 'j_score', 'j_identity', 'j_support', 'fwr1', 'fwr2', 'fwr3', 'fwr4', 'cdr1', 'cdr2', 'cdr3', 'cell_id', 'consensus_count', 'duplicate_count', 'v_call_10x', 'd_call_10x', 'j_call_10x', 'junction_10x', 'junction_10x_aa', 'j_support_igblastn', 'j_score_igblastn', 'j_call_igblastn', 'j_call_blastn', 'j_identity_blastn', 'j_alignment_length_

In [10]:
# library_type is set to filter out genes that are not TRA/TRB (in 'locus' column) as this library is abTCR
tcrab_checked, trab_adata = ddl.pp.check_contigs(tcrab, adata, productive_only = False, library_type = 'tr-ab')
tcrab_checked

Preparing data: 168592it [01:16, 2193.23it/s]
Scanning for poor quality/ambiguous contigs: 100%|██████████| 65177/65177 [09:39<00:00, 112.43it/s]


Dandelion class object with n_obs = 64817 and n_contigs = 168592
    data: 'sequence_id', 'sequence', 'rev_comp', 'productive', 'v_call', 'd_call', 'j_call', 'sequence_alignment', 'germline_alignment', 'junction', 'junction_aa', 'v_cigar', 'd_cigar', 'j_cigar', 'stop_codon', 'vj_in_frame', 'locus', 'c_call', 'junction_length', 'np1_length', 'np2_length', 'v_sequence_start', 'v_sequence_end', 'v_germline_start', 'v_germline_end', 'd_sequence_start', 'd_sequence_end', 'd_germline_start', 'd_germline_end', 'j_sequence_start', 'j_sequence_end', 'j_germline_start', 'j_germline_end', 'v_score', 'v_identity', 'v_support', 'd_score', 'd_identity', 'd_support', 'j_score', 'j_identity', 'j_support', 'fwr1', 'fwr2', 'fwr3', 'fwr4', 'cdr1', 'cdr2', 'cdr3', 'cell_id', 'consensus_count', 'duplicate_count', 'v_call_10x', 'd_call_10x', 'j_call_10x', 'junction_10x', 'junction_10x_aa', 'j_support_igblastn', 'j_score_igblastn', 'j_call_igblastn', 'j_call_blastn', 'j_identity_blastn', 'j_alignment_length_

## tcr-gd

In [11]:
# read in tcr files
trgd_meta = pd.read_csv('metadata/gdTCR_metadata_cleaned_24032022.csv', index_col = 0)
trgd_meta

,GDTCR,GDTCR_sample,GEX file
0,WSSS_F_Imm10424040,F41_LI_45P_GDTCR,FCAImmP7555856
1,WSSS_F_Imm10424041,F41_SK_45P_GDTCR,FCAImmP7555858
2,WSSS_F_Imm10424042,F41_TH_45P_GDTCR,FCAImmP7555860
3,WSSS_F_Imm10424043,F41_BM_45P_GDTCR,FCAImmP7555862
4,WSSS_F_Imm10424044,F45_LI_45P_GDTCR,FCAImmP7579226
5,WSSS_F_Imm10424045,F45_SP_45P_GDTCR,FCAImmP7579228
6,WSSS_F_Imm10424046,F45_TH_45P_GDTCR,FCAImmP7579230
7,WSSS_F_Imm10424047,F45_BM_45P_GDTCR,FCAImmP7579232
8,WSSS_F_Imm10424048,F29_BM_45P_GDTCR,FCAImmP7292031
9,WSSS_F_Imm10424049,F29_SP_45P_GDTCR,FCAImmP7292029


In [12]:
tcrgd = {}
for x, y in tqdm(zip(trgd_meta['GDTCR'], trgd_meta['GEX file'])):
    file = vdj_loc + '/gdT/original/'+x+'/dandelion/tmp/all_contig_igblast_db-all.tsv'
    try:
        tmp = pd.read_csv(file, sep ='\t')
        tmp['cell_id'] = [y + '-' + z.split('-1')[0] for z in tmp['cell_id']]
        tmp = tmp[tmp['cell_id'].isin(adata.obs_names)].copy()
        if 'duplicate_count' not in tmp:
            tmp['duplicate_count'] = tmp['umi_count']
            tmp.drop('umi_count', axis = 1, inplace = True)
        tcrgd[x] = ddl.Dandelion(tmp)
    except:
        # print(x)
        pass
len(tcrgd)

34it [00:31,  1.08it/s]


33

In [13]:
# concatenate
tcrgd = ddl.concat([tcrgd[x] for x in tcrgd], prefixes = list(tcrgd.keys()))
tcrgd

Dandelion class object with n_obs = 21131 and n_contigs = 38945
    data: 'sequence_id', 'sequence', 'rev_comp', 'productive', 'v_call', 'd_call', 'j_call', 'sequence_alignment', 'germline_alignment', 'junction', 'junction_aa', 'v_cigar', 'd_cigar', 'j_cigar', 'stop_codon', 'vj_in_frame', 'locus', 'c_call', 'junction_length', 'np1_length', 'np2_length', 'v_sequence_start', 'v_sequence_end', 'v_germline_start', 'v_germline_end', 'd_sequence_start', 'd_sequence_end', 'd_germline_start', 'd_germline_end', 'j_sequence_start', 'j_sequence_end', 'j_germline_start', 'j_germline_end', 'v_score', 'v_identity', 'v_support', 'd_score', 'd_identity', 'd_support', 'j_score', 'j_identity', 'j_support', 'fwr1', 'fwr2', 'fwr3', 'fwr4', 'cdr1', 'cdr2', 'cdr3', 'cell_id', 'consensus_count', 'duplicate_count', 'v_call_10x', 'd_call_10x', 'j_call_10x', 'junction_10x', 'junction_10x_aa', 'j_support_igblastn', 'j_score_igblastn', 'j_call_igblastn', 'j_call_blastn', 'j_identity_blastn', 'j_alignment_length_b

In [14]:
tcrgd_checked, trgd_adata = ddl.pp.check_contigs(tcrgd, adata, productive_only = False, library_type = 'tr-gd')
tcrgd_checked

Preparing data: 25100it [00:10, 2310.25it/s]
Scanning for poor quality/ambiguous contigs: 100%|██████████| 15526/15526 [01:37<00:00, 159.34it/s]


Dandelion class object with n_obs = 15430 and n_contigs = 25100
    data: 'sequence_id', 'sequence', 'rev_comp', 'productive', 'v_call', 'd_call', 'j_call', 'sequence_alignment', 'germline_alignment', 'junction', 'junction_aa', 'v_cigar', 'd_cigar', 'j_cigar', 'stop_codon', 'vj_in_frame', 'locus', 'c_call', 'junction_length', 'np1_length', 'np2_length', 'v_sequence_start', 'v_sequence_end', 'v_germline_start', 'v_germline_end', 'd_sequence_start', 'd_sequence_end', 'd_germline_start', 'd_germline_end', 'j_sequence_start', 'j_sequence_end', 'j_germline_start', 'j_germline_end', 'v_score', 'v_identity', 'v_support', 'd_score', 'd_identity', 'd_support', 'j_score', 'j_identity', 'j_support', 'fwr1', 'fwr2', 'fwr3', 'fwr4', 'cdr1', 'cdr2', 'cdr3', 'cell_id', 'consensus_count', 'duplicate_count', 'v_call_10x', 'd_call_10x', 'j_call_10x', 'junction_10x', 'junction_10x_aa', 'j_support_igblastn', 'j_score_igblastn', 'j_call_igblastn', 'j_call_blastn', 'j_identity_blastn', 'j_alignment_length_b

## bcr

In [15]:
# read in bcr files
bcr_meta = pd.read_csv('metadata/BCR_metadata_cleaned_07092022_with_donors_merged_2samples.csv')
bcr_meta

,Unnamed: 0,BCR,GEX file,donor
0,0,WSSS_F_Imm10014564,FCAImmP7292029,F29
1,1,WSSS_F_Imm10014565,FCAImmP7292031,F29
2,2,WSSS_F_Imm10014566,FCAImmP7292030,F29
3,3,WSSS_F_Imm10014567,FCAImmP7555860,F41
4,4,WSSS_F_Imm10014568,FCAImmP7555856,F41
5,5,WSSS_F_Imm10014569,FCAImmP7555862,F41
6,6,WSSS_F_Imm10014570,FCAImmP7555859,F41
7,7,WSSS_F_Imm10014571,FCAImmP7579230,F45
8,8,WSSS_F_Imm10014572,FCAImmP7579226,F45
9,9,WSSS_F_Imm10014573,FCAImmP7579228,F45


In [16]:
bcr = {}
for x, y in tqdm(zip(bcr_meta["BCR"], bcr_meta["GEX file"])):
    v_call_g, g_al, g_al_d = {}, {}, {}
    file1 = vdj_loc + "/" + x + "/dandelion/tmp/all_contig_igblast_db-all.tsv"
    file2 = vdj_loc + "/" + x + "/dandelion/all_contig_dandelion.tsv"
    try:
        tmp = ddl.load_data(file1)
        tmp2 = pd.read_csv(file2, sep="\t")
        # update v_call_genotyped and associated columns with data from post-dandelion
        tmp['cell_id'] = [z[:-2] for z in tmp['cell_id']]
        tmp = tmp[tmp["cell_id"].isin(adata.obs_names)].copy()
        if "duplicate_count" not in tmp:
            tmp["duplicate_count"] = tmp["umi_count"]
            tmp.drop("umi_count", axis=1, inplace=True)
        if tmp.shape[0] != 0:
            for _, r in tmp2.iterrows():
                seqid = r.sequence_id
                v_call_g[seqid] = r.v_call_genotyped
            for s in tmp.sequence_id:
                if s not in v_call_g.keys():
                    v_call_g[s] = tmp.loc[s, "v_call"]
            tmp["v_call"] = [v_call_g[si] for si in tmp.sequence_id]
            bcr[x] = ddl.Dandelion(tmp)
    except:
        # print(x)
        pass
len(bcr)

1it [00:06,  6.51s/it]/home/jovyan/dandelion/dandelion/utilities/_utilities.py:587: DtypeWarning: Columns (117,118,119) have mixed types. Specify dtype option on import or set low_memory=False.
16it [00:44,  3.04s/it]/home/jovyan/dandelion/dandelion/utilities/_utilities.py:587: DtypeWarning: Columns (53,73,78,89,90,91,117,118,119) have mixed types. Specify dtype option on import or set low_memory=False.
41it [01:06,  1.62s/it]


40

In [17]:
# concatenate
bcr = ddl.concat([bcr[x] for x in bcr])
bcr

Dandelion class object with n_obs = 25565 and n_contigs = 73366
    data: 'sequence_id', 'sequence', 'rev_comp', 'productive', 'v_call', 'd_call', 'j_call', 'sequence_alignment', 'germline_alignment', 'junction', 'junction_aa', 'v_cigar', 'd_cigar', 'j_cigar', 'stop_codon', 'vj_in_frame', 'locus', 'c_call', 'junction_length', 'np1_length', 'np2_length', 'v_sequence_start', 'v_sequence_end', 'v_germline_start', 'v_germline_end', 'd_sequence_start', 'd_sequence_end', 'd_germline_start', 'd_germline_end', 'j_sequence_start', 'j_sequence_end', 'j_germline_start', 'j_germline_end', 'v_score', 'v_identity', 'v_support', 'd_score', 'd_identity', 'd_support', 'j_score', 'j_identity', 'j_support', 'fwr1', 'fwr2', 'fwr3', 'fwr4', 'cdr1', 'cdr2', 'cdr3', 'cell_id', 'consensus_count', 'duplicate_count', 'v_call_10x', 'd_call_10x', 'j_call_10x', 'junction_10x', 'junction_10x_aa', 'j_support_igblastn', 'j_score_igblastn', 'j_call_igblastn', 'j_call_blastn', 'j_identity_blastn', 'j_alignment_length_b

In [18]:
bcr_checked, bcr_adata = ddl.pp.check_contigs(bcr, adata, productive_only = False, library_type = 'ig')
bcr_checked

Preparing data: 73346it [00:35, 2049.72it/s]
Scanning for poor quality/ambiguous contigs: 100%|██████████| 25565/25565 [04:46<00:00, 89.14it/s] 


Dandelion class object with n_obs = 25505 and n_contigs = 73346
    data: 'sequence_id', 'sequence', 'rev_comp', 'productive', 'v_call', 'd_call', 'j_call', 'sequence_alignment', 'germline_alignment', 'junction', 'junction_aa', 'v_cigar', 'd_cigar', 'j_cigar', 'stop_codon', 'vj_in_frame', 'locus', 'c_call', 'junction_length', 'np1_length', 'np2_length', 'v_sequence_start', 'v_sequence_end', 'v_germline_start', 'v_germline_end', 'd_sequence_start', 'd_sequence_end', 'd_germline_start', 'd_germline_end', 'j_sequence_start', 'j_sequence_end', 'j_germline_start', 'j_germline_end', 'v_score', 'v_identity', 'v_support', 'd_score', 'd_identity', 'd_support', 'j_score', 'j_identity', 'j_support', 'fwr1', 'fwr2', 'fwr3', 'fwr4', 'cdr1', 'cdr2', 'cdr3', 'cell_id', 'consensus_count', 'duplicate_count', 'v_call_10x', 'd_call_10x', 'j_call_10x', 'junction_10x', 'junction_10x_aa', 'j_support_igblastn', 'j_score_igblastn', 'j_call_igblastn', 'j_call_blastn', 'j_identity_blastn', 'j_alignment_length_b

### save

In [19]:
tcrab_checked.write_h5ddl('write/abTCR_110123_panfetal.h5ddl', compression = 'blosc:lz4')
tcrgd_checked.write_h5ddl('write/gdTCR_110123_panfetal.h5ddl', compression = 'blosc:lz4')
bcr_checked.write_h5ddl('write/BCR_110123_panfetal.h5ddl', compression = 'blosc:lz4')
#tcrab_checked.write_h5ddl('write/abTCR_150822_panfetal.h5ddl', compression = 'blosc:lz4')
#tcrgd_checked.write_h5ddl('write/gdTCR_150822_panfetal.h5ddl', compression = 'blosc:lz4')
#bcr_checked.write_h5ddl('write/BCR_150822_panfetal.h5ddl', compression = 'blosc:lz4')

/home/jovyan/dandelion/dandelion/utilities/_core.py:1399: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['sequence_id', 'sequence', 'rev_comp', 'productive', 'v_call', 'd_call',
       'j_call', 'sequence_alignment', 'germline_alignment', 'junction',
       'junction_aa', 'v_cigar', 'd_cigar', 'j_cigar', 'stop_codon',
       'vj_in_frame', 'locus', 'c_call', 'fwr1', 'fwr2', 'fwr3', 'fwr4',
       'cdr1', 'cdr2', 'cdr3', 'cell_id', 'v_call_10x', 'd_call_10x',
       'j_call_10x', 'junction_10x', 'junction_10x_aa', 'j_call_igblastn',
       'j_call_blastn', 'j_sequence_alignment_blastn',
       'j_germline_alignment_blastn', 'cell_id_blastn', 'j_source',
       'd_call_igblastn', 'd_call_blastn', 'd_sequence_alignment_blastn',
       'd_germline_alignment_blastn', 'd_source', 'fwr1_aa', 'fwr2_aa',
       'fwr3_aa', 'fwr4_aa', 'cdr1_aa', 'cdr2_aa', 'cdr3_a

In [7]:
tcrab_checked = ddl.read_h5ddl('write/abTCR_110123_panfetal.h5ddl')
tcrgd_checked = ddl.read_h5ddl('write/gdTCR_110123_panfetal.h5ddl')
bcr_checked = ddl.read_h5ddl('write/BCR_110123_panfetal.h5ddl')

In [20]:
trab_adata.write_h5ad('write/abTCR_110123_panfetal.h5ad', compression = 'gzip')
trgd_adata.write_h5ad('write/gdTCR_110123_panfetal.h5ad', compression = 'gzip')
bcr_adata.write_h5ad('write/BCR_110123_panfetal.h5ad', compression = 'gzip')
#trab_adata.write_h5ad('write/abTCR_150822_panfetal.h5ad', compression = 'gzip')
#trgd_adata.write_h5ad('write/gdTCR_150822_panfetal.h5ad', compression = 'gzip')
#bcr_adata.write_h5ad('write/BCR_150822_panfetal.h5ad', compression = 'gzip')

### make a singular object

In [9]:
tcrgd_checked.data['v_call_genotyped'] = tcrgd_checked.data.v_call
tcrab_checked.data['v_call_genotyped'] = tcrab_checked.data.v_call
vdj = ddl.concat([tcrab_checked,tcrgd_checked,bcr_checked])
vdj # problem that v_call_VDJ_main disappeared

Dandelion class object with n_obs = 93861 and n_contigs = 267038
    data: 'sequence_id', 'sequence', 'rev_comp', 'productive', 'v_call', 'd_call', 'j_call', 'sequence_alignment', 'germline_alignment', 'junction', 'junction_aa', 'v_cigar', 'd_cigar', 'j_cigar', 'stop_codon', 'vj_in_frame', 'locus', 'c_call', 'junction_length', 'np1_length', 'np2_length', 'v_sequence_start', 'v_sequence_end', 'v_germline_start', 'v_germline_end', 'd_sequence_start', 'd_sequence_end', 'd_germline_start', 'd_germline_end', 'j_sequence_start', 'j_sequence_end', 'j_germline_start', 'j_germline_end', 'v_score', 'v_identity', 'v_support', 'd_score', 'd_identity', 'd_support', 'j_score', 'j_identity', 'j_support', 'fwr1', 'fwr2', 'fwr3', 'fwr4', 'cdr1', 'cdr2', 'cdr3', 'cell_id', 'consensus_count', 'duplicate_count', 'v_call_10x', 'd_call_10x', 'j_call_10x', 'junction_10x', 'junction_10x_aa', 'j_support_igblastn', 'j_score_igblastn', 'j_call_igblastn', 'j_call_blastn', 'j_identity_blastn', 'j_alignment_length_

In [10]:
ddl.tl.transfer(adata, vdj)

In [11]:
adata.write_h5ad('write/PAN.A01.v01.entire_data_normalised_log.20210429.LYMPHOID.embedding_dandelion_110123.h5ad', compression = 'gzip')
#adata.write_h5ad('write/PAN.A01.v01.entire_data_normalised_log.20210429.LYMPHOID.embedding_dandelion_150822.h5ad', compression = 'gzip')

In [12]:
adata

AnnData object with n_obs × n_vars = 241950 × 33538
    obs: 'n_counts', 'n_genes', 'file', 'mito', 'doublet_scores', 'predicted_doublets', 'name', 'uniform_label', 'uniform_label_expanded_merged', 'uniform_label_lvl0', 'organ', 'Sample.lanes', 'Sort_id', 'age', 'method', 'donor', 'sex', 'Processing_method', 'AnnatomicalPart', 'Sample', 'bbk', 'scvi_clusters', 'anno_lvl_2_LYMPHOID', 'anno_lvl_2_MYELOID', 'anno_lvl_2_MEM_PROGENITORS', 'leiden_150', 'anno_lvl_2_final_clean', 'sample_id', 'locus_VDJ', 'locus_VJ', 'productive_VDJ', 'productive_VJ', 'v_call_genotyped_VDJ', 'd_call_VDJ', 'j_call_VDJ', 'v_call_genotyped_VJ', 'j_call_VJ', 'c_call_VDJ', 'c_call_VJ', 'junction_VDJ', 'junction_VJ', 'junction_aa_VDJ', 'junction_aa_VJ', 'v_call_genotyped_B_VDJ', 'd_call_B_VDJ', 'j_call_B_VDJ', 'v_call_genotyped_B_VJ', 'j_call_B_VJ', 'c_call_B_VDJ', 'c_call_B_VJ', 'productive_B_VDJ', 'productive_B_VJ', 'v_call_genotyped_abT_VDJ', 'd_call_abT_VDJ', 'j_call_abT_VDJ', 'v_call_genotyped_abT_VJ', 'j_call